In [2]:
# import file as rdd
file = sc.textFile('/home/lee/Downloads/NASA_access_log_Jul95.txt')
# grab top 10
file.take(10)

['199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245',
 'unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985',
 '199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085',
 'burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0',
 '199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179',
 'burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 304 0',
 'burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/video/livevideo.gif HTTP/1.0" 200 0',
 '205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/countdown.html HTTP/1.0" 200 3985',
 'd104.aa.net - - [01/Jul/1995:00:00:13 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985',
 '129.94.144.152 - - [01/Jul/

In [3]:
# get total count
file.count()

1891715

In [18]:
# turn into CSV format and split into columns, remove any records where columns are empty
parsed_logs = file.map(lambda l: l \
        .replace(' - - [', ',') \
        .replace('] "', ',') \
        .replace('" ', ',')
        .replace(' ', ',')
        .split(',')) \
.filter(lambda l: len(l) == 8)

# get total count after filter
parsed_logs.count()

In [20]:
# check type
type(parsed_logs)

pyspark.rdd.PipelinedRDD

In [22]:
# convert rdd to dataframe
df_parsed_logs = parsed_logs.toDF()
# check type
type(df_parsed_logs)

pyspark.sql.dataframe.DataFrame

In [23]:
# get top row
df_parsed_logs.head()

Row(_1='199.72.81.55', _2='01/Jul/1995:00:00:01', _3='-0400', _4='GET', _5='/history/apollo/', _6='HTTP/1.0', _7='200', _8='6245')

In [24]:
# validate distinct count
df_parsed_logs.select('_1').distinct().count()

81885

In [30]:
# get distinct IP/Host names
distinct_logs = df_parsed_logs.select('_1').distinct()
distinct_logs.coalesce(1).write.csv('/home/lee/Downloads/NASA_Distinct')
distinct_logs.show()

+--------------------+
|                  _1|
+--------------------+
|ppp3_136.bekkoame...|
|    ppp31.texoma.com|
|ix-wc7-20.ix.netc...|
|nb1-du5.polarnet....|
|    ttyb5.shasta.com|
|dialup00004.cinet...|
|dd14-025.compuser...|
|nigrlpr.actrix.ge...|
| uckm001.pn.itnet.it|
|      queulen.puc.cl|
|pipe2.nyc.pipelin...|
|      198.53.164.131|
|asdsun.larc.nasa.gov|
|pm45-52.smartlink...|
|     193.166.184.116|
|       194.20.34.120|
|dd09-021.compuser...|
|     leo.racsa.co.cr|
| freenet.carleton.ca|
|  enigma.idirect.com|
+--------------------+
only showing top 20 rows



In [31]:
# get request count for all
df_parsed_logs.groupBy('_4').count().show()

+----+-------+
|  _4|  count|
+----+-------+
|POST|    111|
|HEAD|   3945|
| GET|1844926|
+----+-------+



In [53]:
# convert column from string to int
df_parsed_logs = df_parsed_logs.withColumn('_8', df_parsed_logs['_8'].cast('Integer'))

# check
#df_parsed_logs.select('_8').dtypes

# get IP and bytes sum count
groupBy_logs = df_parsed_logs.groupBy('_1').sum('_8')
groupBy_logs.coalesce(1).write.csv('/home/lee/Downloads/NASA_GroupBy')
groupBy_logs.show()

+--------------------+--------+
|                  _1| sum(_8)|
+--------------------+--------+
|ppp3_136.bekkoame...|  224403|
|    ppp31.texoma.com| 1486571|
|ix-wc7-20.ix.netc...|  406994|
|nb1-du5.polarnet....|  409095|
|    ttyb5.shasta.com|  138918|
|dialup00004.cinet...|   72814|
|dd14-025.compuser...|  760617|
|nigrlpr.actrix.ge...|   88557|
| uckm001.pn.itnet.it|  349801|
|      queulen.puc.cl| 1753836|
|pipe2.nyc.pipelin...| 2283246|
|      198.53.164.131|   76930|
|asdsun.larc.nasa.gov|  193436|
|pm45-52.smartlink...|    7918|
|     193.166.184.116|       0|
|       194.20.34.120|  602948|
|dd09-021.compuser...|   58781|
|     leo.racsa.co.cr|    3980|
| freenet.carleton.ca|   23081|
|  enigma.idirect.com|22370563|
+--------------------+--------+
only showing top 20 rows



In [54]:
# get IP and Request count
groupByMulti_logs = df_parsed_logs.groupBy('_1','_4').count()
groupByMulti_logs.coalesce(1).write.csv('/home/lee/Downloads/NASA_GroupByMulti')
groupByMulti_logs.show()

+--------------------+---+-----+
|                  _1| _4|count|
+--------------------+---+-----+
|sam-slip-l6.neoso...|GET|    5|
|           198.6.9.2|GET|   12|
|dial3-10.midwest.net|GET|   13|
|  mill2.millcomm.com|GET|  167|
|      198.59.100.170|GET|   14|
|      srf-36.nbn.com|GET|    9|
|sbd0124.deltanet.com|GET|   11|
|  miafl2-19.gate.net|GET|   25|
|  hplabs.nbnet.nb.ca|GET|    7|
|ibbeta18.ppp.usit...|GET|    9|
|ppp056-stdkn2.ula...|GET|   78|
|gr01.ppp.alliance...|GET|    2|
|ad03-027.compuser...|GET|   79|
| comserv-d-1.usc.edu|GET|   23|
|   bhbwin.acadia.net|GET|   26|
|     roy.slip.lm.com|GET|   35|
|        biron.mv.com|GET|    2|
|  dialup57.aloha.com|GET|    3|
|dd12-011.compuser...|GET|   31|
|    ttyq0.tyrell.net|GET|   26|
+--------------------+---+-----+
only showing top 20 rows

